# Setup
This section contains all necessary setup such as libraries and model creation.

In [1]:
import tensorflow as tf
import pandas as pd

from src.data.data_helper import get_raw_data_as_dataframe
from src.models.preprocessing.preprocessor import calibrate_pre_processor, pre_processor_layer

In [2]:
raw_data = get_raw_data_as_dataframe()
raw_data.head()

,time,measurement,label,source
0,0.0000,1.610419,0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
1,0.0002,1.610419,0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
2,0.0004,1.610419,0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
3,0.0006,1.610419,0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
4,0.0008,1.610419,0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...


## Pre-processor Calibration
Due to the fact that all data needs to be pre-processed using the same bandpass filter and normalization the pre-processor needs to be calibrated. This is done by calculating the coefficients of the bandpass filter and the normalization parameters.

These parameters will then be passed to the selected model class.

In [3]:
a, b, variance = calibrate_pre_processor(raw_data)

## Data Parsing
This section deals with segmenting the data into usable segements with correct labeling.

In [4]:
from src.data.data_helper import segement_data

In [5]:
segmented_data = segement_data(raw_data, window_length=200*5, overlap=50*5)

In [6]:
segmented_data.head()

,window_data,label,source
0,"[1.6104191404439578, 1.6104191404439578, 1.610...",0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
1,"[1.6288106990610434, 1.6288106990610434, 1.628...",0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
2,"[1.6324890107844603, 1.6324890107844603, 1.628...",0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
3,"[1.6361673225078774, 1.6361673225078774, 1.636...",0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
4,"[1.6251323873376262, 1.6251323873376262, 1.625...",0,C:\Users\erik\IES_codebase\EMG_Project\CDT406-...


In [7]:
segmented_data.iloc[50]

window_data    [1.4007553722091834, 1.411790307379435, 1.4265...
label                                                          2
source         C:\Users\erik\IES_codebase\EMG_Project\CDT406-...
Name: 50, dtype: object

In [8]:
num_classes = segmented_data['label'].nunique()
num_classes

4

In [9]:
y_data = np.array(segmented_data['label'].values)
X_data = segmented_data.drop(columns=['label', 'source'])

NameError: name 'np' is not defined

In [60]:
y_data.shape

(9930,)

,window_data
0,"[1.6104191404439578, 1.6104191404439578, 1.610..."
1,"[1.6288106990610434, 1.6288106990610434, 1.628..."
2,"[1.6324890107844603, 1.6324890107844603, 1.628..."
3,"[1.6361673225078774, 1.6361673225078774, 1.636..."
4,"[1.6251323873376262, 1.6251323873376262, 1.625..."
...,...
9925,"[1.6251323873376262, 1.6251323873376262, 1.625..."
9926,"[1.65823719284838, 1.654558881124963, 1.654558..."
9927,"[1.621454075614209, 1.621454075614209, 1.62513..."
9928,"[1.691341998359134, 1.691341998359134, 1.68766..."


In [40]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
        X_data, y_data, test_size=0.2
    )

In [54]:
X_train[1]

array([array([1.87157927, 1.87525758, 1.8789359 , 1.8789359 , 1.8789359 ,
              1.88261421, 1.88261421, 1.88261421, 1.88261421, 1.8789359 ,
              1.8789359 , 1.87525758, 1.87157927, 1.87157927, 1.86422265,
              1.86054434, 1.85686603, 1.85318771, 1.84583109, 1.84215278,
              1.83847447, 1.83479616, 1.83111784, 1.82743953, 1.82008291,
              1.8164046 , 1.81272629, 1.80536966, 1.79801304, 1.79433473,
              1.79065641, 1.7869781 , 1.77962148, 1.77594317, 1.77226486,
              1.76490823, 1.76490823, 1.76122992, 1.75755161, 1.75755161,
              1.7538733 , 1.75019499, 1.75019499, 1.75019499, 1.74651667,
              1.74651667, 1.74651667, 1.74651667, 1.74283836, 1.74283836,
              1.73916005, 1.73916005, 1.73916005, 1.73548174, 1.73180343,
              1.73180343, 1.72812512, 1.72812512, 1.7244468 , 1.72076849,
              1.72076849, 1.71709018, 1.71341187, 1.70973356, 1.70605525,
              1.70605525, 1.70237693, 

## Model Setup

In [9]:
from src.models.LSTM.LSTM import LSTM
from src.models.LSTM_STFT.LSTM_STFT import LSTM_STFT
from src.models.LSTM_STFT_Dense.LSTM_STFT_Dense import LSTM_STFT_Dense

In [38]:
model = LSTM(input_shape=X_data.shape[1], num_classes=num_classes, a=a, b=b, variance=variance)

In [50]:
from tensorflow.keras import layers, models

model = models.Sequential([
            layers.Input(shape=(X_data.shape[1],)),
            # layers.Lambda(lambda x: pre_processor_layer(x, b, a, variance), name='pre_processor'),
            # layers.Reshape((1, -1)),
            layers.Dense(32, activation='tanh'), # Based on the architecture of the paper
            layers.LSTM(64, unroll=True, activation='tanh'),
            layers.Dense(32, activation='tanh'),
            layers.Dense(num_classes, activation='softmax')
        ])

In [51]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train Model

In [52]:
model.fit(X_train, y_train,
                       validation_data=(X_val, y_val),
                       epochs=10,
                       batch_size=32,
                       verbose=1)

ValueError: Invalid dtype: object

In [41]:
model.train(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, epochs=10)

ValueError: Invalid dtype: object